In [1]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
spark=SparkSession.builder.master("spark://10.182.1.163:7077").appName('sample-data').getOrCreate()

22/01/28 11:57:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/28 11:57:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
spark

In [3]:
# dataframe
df=spark.read.csv("/root/Downloads/Malicious_Domain_Name_Dataset_10000.csv",header=True,inferSchema=True)
df

DataFrame[domain_name: string, asn: int, ip: string, owner: string, country_code: string, malware: int, domain _length: int, n_letters: int, n_number: int, n_hypans: int, n_dots: int, entropy: double]

In [4]:
df.count()

10000

In [5]:
df.printSchema()

root
 |-- domain_name: string (nullable = true)
 |-- asn: integer (nullable = true)
 |-- ip: string (nullable = true)
 |-- owner: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- malware: integer (nullable = true)
 |-- domain _length: integer (nullable = true)
 |-- n_letters: integer (nullable = true)
 |-- n_number: integer (nullable = true)
 |-- n_hypans: integer (nullable = true)
 |-- n_dots: integer (nullable = true)
 |-- entropy: double (nullable = true)



In [6]:
df.limit(5).toPandas()

,domain_name,asn,ip,owner,country_code,malware,domain _length,n_letters,n_number,n_hypans,n_dots,entropy
0,netflix.com,16509,52.49.120.6,"AMAZON-02 - Amazon.com, Inc.",US,0,11,10,0,0,1,3.459432
1,push.prod.netflix.com,16509,35.167.71.134,"AMAZON-02 - Amazon.com, Inc.",US,0,21,18,0,0,3,3.975418
2,api-global.netflix.com,16509,52.42.194.172,"AMAZON-02 - Amazon.com, Inc.",US,0,22,19,0,1,2,3.879664
3,google.com,15169,74.125.68.138,GOOGLE - Google LLC,US,0,10,9,0,0,1,2.646439
4,ftl.netflix.com,40027,45.57.40.1,NETFLIX-ASN - Netflix Streaming Services Inc.,US,0,15,13,0,0,2,3.373557


In [7]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct(df['domain_name'])).show()

+---------------------------+
|count(DISTINCT domain_name)|
+---------------------------+
|                       9990|
+---------------------------+



In [8]:
df.select(countDistinct(df['owner'])).show()

+---------------------+
|count(DISTINCT owner)|
+---------------------+
|                 1102|
+---------------------+



In [9]:
df=df.withColumnRenamed('malware','label')

In [10]:
df.limit(5).toPandas()

,domain_name,asn,ip,owner,country_code,label,domain _length,n_letters,n_number,n_hypans,n_dots,entropy
0,netflix.com,16509,52.49.120.6,"AMAZON-02 - Amazon.com, Inc.",US,0,11,10,0,0,1,3.459432
1,push.prod.netflix.com,16509,35.167.71.134,"AMAZON-02 - Amazon.com, Inc.",US,0,21,18,0,0,3,3.975418
2,api-global.netflix.com,16509,52.42.194.172,"AMAZON-02 - Amazon.com, Inc.",US,0,22,19,0,1,2,3.879664
3,google.com,15169,74.125.68.138,GOOGLE - Google LLC,US,0,10,9,0,0,1,2.646439
4,ftl.netflix.com,40027,45.57.40.1,NETFLIX-ASN - Netflix Streaming Services Inc.,US,0,15,13,0,0,2,3.373557


In [11]:
# checking for classes
df.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1| 5000|
|    0| 5000|
+-----+-----+



In [12]:
df.toPandas().isnull().sum()

22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 714
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 714
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 715
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 715
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 716
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 716
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 717
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 717
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 718
22/01/28 11:58:25 WARN AccumulatorContext: Attempted to access garbage collected accumulator 718
22/01/28 11:58:25 WARN Accumul

domain_name       0
asn               0
ip                0
owner             0
country_code      0
label             0
domain _length    0
n_letters         0
n_number          0
n_hypans          0
n_dots            0
entropy           0
dtype: int64

In [13]:
df.toPandas().describe()

,asn,label,domain _length,n_letters,n_number,n_hypans,n_dots,entropy
count,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,41165.383800,0.500000,18.702300,15.92710,0.673400,0.245400,1.856300,3.430663
std,59746.806254,0.500025,7.985653,6.55775,1.871706,0.671136,0.882229,0.367327
min,2.000000,0.000000,4.000000,2.00000,0.000000,0.000000,1.000000,1.664498
25%,14618.000000,0.000000,13.000000,11.00000,0.000000,0.000000,1.000000,3.189898
50%,20473.000000,0.500000,17.000000,15.00000,0.000000,0.000000,2.000000,3.447402
75%,40034.000000,1.000000,22.000000,19.00000,0.000000,0.000000,2.000000,3.669350
max,396190.000000,1.000000,126.000000,94.00000,66.000000,9.000000,15.000000,4.821105


In [14]:
categorical_cols=[item[0] for item in df.dtypes if item[1].startswith('string')]
categorical_cols

['domain_name', 'ip', 'owner', 'country_code']

In [15]:
numerical_cols=[item[0] for item in df.dtypes if item[1].startswith('int')]
numerical_cols

['asn',
 'label',
 'domain _length',
 'n_letters',
 'n_number',
 'n_hypans',
 'n_dots']

In [16]:
numerical_data=df.select(numerical_cols).toPandas()
numerical_data

,asn,label,domain _length,n_letters,n_number,n_hypans,n_dots
0,16509,0,11,10,0,0,1
1,16509,0,21,18,0,0,3
2,16509,0,22,19,0,1,2
3,15169,0,10,9,0,0,1
4,40027,0,15,13,0,0,2
...,...,...,...,...,...,...,...
9995,46606,1,23,22,0,0,1
9996,53831,1,16,15,0,0,1
9997,394695,1,14,12,0,0,2
9998,26496,1,13,12,0,0,1


In [17]:
numerical_data.corr()

,asn,label,domain _length,n_letters,n_number,n_hypans,n_dots
asn,1.000000,0.313619,0.040611,0.008990,0.171192,-0.026530,-0.042217
label,0.313619,1.000000,-0.110867,-0.081297,0.026822,-0.146028,-0.345166
domain _length,0.040611,-0.110867,1.000000,0.957067,0.423308,0.510038,0.650882
n_letters,0.008990,-0.081297,0.957067,1.000000,0.169021,0.404751,0.562784
n_number,0.171192,0.026822,0.423308,0.169021,1.000000,0.278054,0.241818
n_hypans,-0.026530,-0.146028,0.510038,0.404751,0.278054,1.000000,0.256680
n_dots,-0.042217,-0.345166,0.650882,0.562784,0.241818,0.256680,1.000000


In [18]:
#removing domain_length col
df=df.drop('domain _length')

In [19]:
df.toPandas()

,domain_name,asn,ip,owner,country_code,label,n_letters,n_number,n_hypans,n_dots,entropy
0,netflix.com,16509,52.49.120.6,"AMAZON-02 - Amazon.com, Inc.",US,0,10,0,0,1,3.459432
1,push.prod.netflix.com,16509,35.167.71.134,"AMAZON-02 - Amazon.com, Inc.",US,0,18,0,0,3,3.975418
2,api-global.netflix.com,16509,52.42.194.172,"AMAZON-02 - Amazon.com, Inc.",US,0,19,0,1,2,3.879664
3,google.com,15169,74.125.68.138,GOOGLE - Google LLC,US,0,9,0,0,1,2.646439
4,ftl.netflix.com,40027,45.57.40.1,NETFLIX-ASN - Netflix Streaming Services Inc.,US,0,13,0,0,2,3.373557
...,...,...,...,...,...,...,...,...,...,...,...
9995,californiaroaddream.com,46606,69.89.31.98,UNIFIEDLAYER-AS-1 - Unified Layer,US,1,22,0,0,1,3.414441
9996,callahanward.com,53831,198.185.159.144,"SQUARESPACE - Squarespace, Inc.",US,1,15,0,0,1,3.250000
9997,callisto.co.in,394695,199.79.62.87,PUBLIC-DOMAIN-REGISTRY - PDR,US,1,12,0,0,2,3.093069
9998,caloaksgc.com,26496,184.168.131.241,"AS-26496-GO-DADDY-COM-LLC - GoDaddy.com, LLC",US,1,12,0,0,1,3.026987


In [20]:
new_numerical_cols=[item[0] for item in df.dtypes if item[1].startswith('int')]
new_numerical_cols

['asn', 'label', 'n_letters', 'n_number', 'n_hypans', 'n_dots']

In [21]:
n_numerical_data=df.select(new_numerical_cols).toPandas()


In [22]:
n_numerical_data.corr()

,asn,label,n_letters,n_number,n_hypans,n_dots
asn,1.000000,0.313619,0.008990,0.171192,-0.026530,-0.042217
label,0.313619,1.000000,-0.081297,0.026822,-0.146028,-0.345166
n_letters,0.008990,-0.081297,1.000000,0.169021,0.404751,0.562784
n_number,0.171192,0.026822,0.169021,1.000000,0.278054,0.241818
n_hypans,-0.026530,-0.146028,0.404751,0.278054,1.000000,0.256680
n_dots,-0.042217,-0.345166,0.562784,0.241818,0.256680,1.000000


In [23]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier

In [24]:
#train_test_split
train_data,test_data=df.randomSplit([0.8,0.2])

In [25]:
train_data.count()

7994

In [26]:
test_data.count()

2006

In [27]:
# checking for classes
import pyspark.sql.functions as F
from pyspark.sql.types import *

tot = train_data.count()

tf=train_data.groupby('label').count().withColumnRenamed('count', 'cnt_per_group') \
  .withColumn('perc_of_count_total', (F.col('cnt_per_group') / tot) * 100 ).show()

+-----+-------------+-------------------+
|label|cnt_per_group|perc_of_count_total|
+-----+-------------+-------------------+
|    1|         3975| 49.724793595196395|
|    0|         4019| 50.275206404803605|
+-----+-------------+-------------------+



In [28]:
# checking for classes
import pyspark.sql.functions as F
from pyspark.sql.types import *

tot1 = test_data.count()

testf=test_data.groupby('label').count().withColumnRenamed('count', 'cnt_per_group') \
  .withColumn('perc_of_count_total', (F.col('cnt_per_group') / tot1) * 100 ).show()

+-----+-------------+-------------------+
|label|cnt_per_group|perc_of_count_total|
+-----+-------------+-------------------+
|    1|         1025| 51.096709870388835|
|    0|          981| 48.903290129611165|
+-----+-------------+-------------------+



In [29]:
# define stage 1: transform the column feature_2 to numeric
stage_1 = StringIndexer(inputCol= 'domain_name', outputCol= 'domain_name_index', handleInvalid = "keep")

# define stage 2: transform the column feature_3 to numeric
stage_2 = StringIndexer(inputCol= 'ip', outputCol= 'ip_index', handleInvalid = "keep")

# define stage 2: transform the column feature_3 to numeric
stage_3 = StringIndexer(inputCol= 'owner', outputCol= 'owner_index', handleInvalid = "keep")

# define stage 2: transform the column feature_3 to numeric
stage_4 = StringIndexer(inputCol= 'country_code', outputCol= 'country_code_index', handleInvalid = "keep")


# define stage 5: one hot encode the numeric versions of feature 1,2,3 and 4 generated from stage 1,2,3 and stage 4
stage_5 = OneHotEncoder(inputCols=[stage_1.getOutputCol(), stage_2.getOutputCol(),stage_3.getOutputCol(),stage_4.getOutputCol()], 
                                 outputCols= ['domain_name_encoded', 'ip_encoded','owner_encoded','country_code_encoded'])
# define stage 4: create a vector of all the features required to train the logistic regression model 
stage_6 = VectorAssembler(inputCols=['domain_name_encoded', 'asn', 'ip_encoded', 'owner_encoded','country_code_encoded','n_letters','n_number','n_hypans','n_dots','entropy'],
                          outputCol='features')
# define stage 5: logistic regression model                          
stage_7 = RandomForestClassifier(featuresCol='features',labelCol='label',numTrees=200)

In [30]:
# setup the pipeline
rf_classification_pipeline = Pipeline(stages= [stage_1, 
                                        stage_2, 
                                        stage_3, 
                                        stage_4, 
                                        stage_5,
                                        stage_6,
                                        stage_7])

In [31]:
# training
model = rf_classification_pipeline.fit(train_data)

22/01/28 11:59:05 WARN DAGScheduler: Broadcasting large task binary with size 1924.3 KiB
22/01/28 11:59:06 WARN DAGScheduler: Broadcasting large task binary with size 1924.4 KiB
22/01/28 11:59:07 WARN DAGScheduler: Broadcasting large task binary with size 1930.4 KiB
22/01/28 11:59:08 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/01/28 11:59:14 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
22/01/28 11:59:19 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/01/28 11:59:23 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/01/28 11:59:29 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [32]:
# Make predictions.
predictions = model.transform(test_data)



In [36]:
model.stages #train data

[StringIndexerModel: uid=StringIndexer_fa7cab01cda3, handleInvalid=keep,
 StringIndexerModel: uid=StringIndexer_21eb0d4ce833, handleInvalid=keep,
 StringIndexerModel: uid=StringIndexer_0d22a2279281, handleInvalid=keep,
 StringIndexerModel: uid=StringIndexer_05b1c533d2dd, handleInvalid=keep,
 OneHotEncoderModel: uid=OneHotEncoder_0d5ff0ac767c, dropLast=true, handleInvalid=error, numInputCols=4, numOutputCols=4,
 VectorAssembler_ee7ea76dea00,
 RandomForestClassificationModel: uid=RandomForestClassifier_e80bf30a7802, numTrees=200, numClasses=2, numFeatures=14803]

In [37]:
m_summary=model.stages[6].summary

In [39]:
m_summary

22/02/09 11:36:47 WARN TransportChannelHandler: Exception in connection from /10.182.1.163:7077
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:148)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:714)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:650)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.java

In [38]:
m_summary.accuracy

22/01/28 12:01:34 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


0.8660245183887916

In [46]:
m_summary.precisionByLabel

[0.8732323232323232, 0.8675642594859241]

In [ ]:
p=tp/(tp+fp)

In [ ]:
r=tp/(tp+fn)

In [47]:
m_summary.recallByLabel            0.86= lable1 0.87=lable2

[0.8647161790447612, 0.8759268413247652]

In [48]:
#train data
m_summary.areaUnderROC

0.9328079980944322

In [49]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(15)



+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       1.0|    1|(14886,[8037,8038...|
|       1.0|    1|(14886,[8037,1459...|
|       1.0|    1|(14886,[8037,1396...|
|       1.0|    1|(14886,[8037,1403...|
|       1.0|    1|(14886,[8037,1395...|
|       1.0|    1|(14886,[8037,8038...|
|       1.0|    0|(14886,[8037,1425...|
|       1.0|    1|(14886,[8037,8075...|
|       1.0|    1|(14886,[8037,1396...|
|       1.0|    1|(14886,[8037,8038...|
|       0.0|    0|(14886,[8037,1382...|
|       0.0|    1|(14886,[8037,1288...|
|       0.0|    1|(14886,[8037,8045...|
|       0.0|    0|(14886,[8037,1383...|
|       0.0|    0|(14886,[8037,8226...|
+----------+-----+--------------------+
only showing top 15 rows



In [50]:
# see the prediction on the test data
predictions.select('probability', 'prediction').show()


+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.48148163280169...|       1.0|
|[0.46266718518431...|       1.0|
|[0.48139621854574...|       1.0|
|[0.49396195246891...|       1.0|
|[0.46139313638858...|       1.0|
|[0.46969174605488...|       1.0|
|[0.46849132102003...|       1.0|
|[0.47846998779419...|       1.0|
|[0.47817483936608...|       1.0|
|[0.47344232841864...|       1.0|
|[0.53827185775710...|       0.0|
|[0.53176753523985...|       0.0|
|[0.52057931647178...|       0.0|
|[0.54827455093298...|       0.0|
|[0.50316984514231...|       0.0|
|[0.55366738998152...|       0.0|
|[0.48481678800112...|       1.0|
|[0.46387745508603...|       1.0|
|[0.50233555146083...|       0.0|
|[0.54739607706168...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [51]:
#test data
# Select (prediction, true label) and compute test error
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
auc = evaluator.evaluate(predictions)

rfModel = model.stages[2]
print(rfModel)  # summary o

StringIndexerModel: uid=StringIndexer_af1fa31802bf, handleInvalid=keep


In [52]:
print(str(auc))

0.8732938968788024


In [53]:


from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval_f1 = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
f1_score=eval_f1.evaluate(predictions)
print(f1_score)

0.8731631092095513


In [54]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

eval_accu = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
accuracy=eval_accu.evaluate(predictions)
print(accuracy)

0.8731457800511508


#### Cross Validation with hyperparameter grid search

In [55]:
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np

paramGrid = ParamGridBuilder() \
    .addGrid(stage_7.numTrees, [int(x) for x in np.linspace(start = 10, stop = 100, num = 4)]) \
    .addGrid(stage_7.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
    .build()

In [56]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

crossval = CrossValidator(estimator=rf_classification_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)

In [7]:
spark.stop()